In [35]:
import re
import csv
import math
import json
import numpy as np
import pandas as pd
import pickle
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
import lyricsgenius
genius_TOKEN = 'sD0C3epnJdfOQQK4eIC45dHl-Qv7DipToGpuj1n4WeuG5_LDP1HKn31w5Cn1lOux'
genius = lyricsgenius.Genius(genius_TOKEN)
genius.verbose = False
genius.remove_section_headers = True

In [37]:
movies = pd.read_csv('merged_data.csv')

In [38]:
def listify(df):
    genres = []
    languages = []
    countries = []
    for x,y,z in zip(df['Genres'],df['Languages'],df['Countries']):
        g = re.findall(': \"(.*?)\"', x)
        l = re.findall(': \"(.*?)\"', y)
        c = re.findall(': \"(.*?)\"', z)
        genres.append(g)
        languages.append(l)
        countries.append(c)
    df['Genres'] = genres
    df['Languages'] = languages
    df['Countries'] = countries
    return df

In [39]:
#movies = listify(movies)

In [55]:
'''years = []
for x in movies['ReleaseDate']:
    x = str(x)[:4]
    if x != 'nan':
        years.append(int(x))
    else:
        years.append(3000)
movies['ReleaseDate'] = years'''

In [44]:
#movies = movies[movies['ReleaseDate']>=1960]  

In [40]:
vectorizer = TfidfVectorizer(max_features=50000, max_df=0.8, min_df=20, norm='l2')
tokenizer = vectorizer.build_tokenizer()

In [41]:
movies['toks'] = [tokenizer(summary) for summary in movies['Summary']]

In [56]:
is_english = ['English Language' in m for m in movies['Languages']]
movies = movies[is_english]

In [58]:
movies = movies.reset_index()
movies = movies.drop(['index'], axis=1)

In [59]:
movies.head()

,WikiID,Title,ReleaseDate,Runtime,Languages,Countries,Genres,Summary,compound,pos,neg,neu,toks
0,975900,Ghosts of Mars,2001,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th...",-0.9913,0.065,0.150,0.786,"[Set, in, the, second, half, of, the, 22nd, ce..."
1,9363483,White Of The Eye,1987,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...,-0.9985,0.078,0.198,0.724,"[series, of, murders, of, rich, young, women, ..."
2,18998739,The Sorcerer's Apprentice,2002,86.0,[English Language],[South Africa],"[Family Film, Fantasy, Adventure, World cinema]","Every hundred years, the evil Morgana returns...",-0.8885,0.092,0.131,0.776,"[Every, hundred, years, the, evil, Morgana, re..."
3,6631279,Little city,1997,93.0,[English Language],[United States of America],"[Romantic comedy, Ensemble Film, Comedy-drama,...","Adam, a San Francisco-based artist who works a...",-0.7097,0.089,0.108,0.804,"[Adam, San, Francisco, based, artist, who, wor..."
4,171005,Henry V,1989,137.0,[English Language],[United Kingdom],"[Costume drama, War film, Epic, Period piece, ...",{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,0.3182,0.065,0.032,0.903,"[Plot, dateAct, 1Act, 2Act, 3Act, 4Act, Finall..."


In [60]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [61]:
s = genius.search_song('XO','eden')
x = analyzer.polarity_scores(s.lyrics)
x1 = x['pos']
y1 = x['neg']

In [62]:
dist = []
for x2,y2 in tuple(zip(movies['pos'], movies['neg'])):
    dist.append(math.sqrt((x1 - x2)**2 + (y1 - y2)**2))

In [63]:
num_movies = len(movies)
num_movies

24775

In [65]:
def build_inverted_index(movies):
    word_set = []
    for t in movies['toks']:
        word_set+=t
    word_set = set(word_set)
    word_dict = {w: [] for w in word_set}
    for i in range(len(movies)):
        for w in set(movies['toks'][i]):
            if w in movies['toks'][i]:
                word_dict[w].append((i,movies['toks'][i].count(w)))
    return word_dict

In [66]:
#took long time to run, only needs to be done once
idx = build_inverted_index(movies)

In [ ]:
np.save('inv_idx.npy', idx) 

In [67]:
df = pd.DataFrame.from_dict(idx,orient='index')

MemoryError: Unable to allocate 21.1 GiB for an array with shape (117649, 24054) and data type object

In [ ]:
#inv_idx = np.load('inv_idx.npy',allow_pickle='TRUE').item()

In [30]:
#with open('inv_idx.txt', 'w') as file:
#    json.dump(idx, file)

In [ ]:
with open('app/inv_idx.txt', 'r') as file:
    inv_idx = json.load(file)

In [28]:
with open('inv_idx.csv', 'w', encoding='utf-8') as f: 
    l = []
    for k, v in idx.items():
        l.append('%s: %s' % (str(k), str(v)))
    f.write(', '.join(l)) 

In [29]:
with open('inv_idx.csv', 'r', encoding='utf-8') as f:
    inv_idx = {}
    l = f.read().split(',')
    for i in l:
        values = i.split(': ')
        inv_idx[values[0]] = values[1]

IndexError: list index out of range

In [37]:
#with open('inv_idx.pkl', 'wb') as f:
#    pickle.dump(inv_idx, f, pickle.HIGHEST_PROTOCOL)

In [41]:
#with open('inv_idx.txt', 'r') as file:
#    new_d = json.load(file)

In [23]:
def compute_idf(inv_idx, n_docs, min_df=20, max_df_ratio=0.8):
    idf = {x: math.log2(n_docs/(1+len(inv_idx[x]))) for x in inv_idx 
           if len(inv_idx[x])>=min_df and len(inv_idx[x])/n_docs<=max_df_ratio}
    return idf

In [24]:
idf = compute_idf(inv_idx,num_movies)

In [25]:
def compute_doc_norms(index, idf, n_docs):
    norms_sq = np.zeros(n_docs)
    for t in idf:
        for (doc,cnt) in index[t]:
            norms_sq[doc] += (cnt*idf[t])**2
    return np.sqrt(norms_sq)

In [26]:
#norms = compute_doc_norms(inv_idx, idf, num_movies)
#np.savetxt('norms.csv', norms, delimiter=',')

In [27]:
norms = np.loadtxt('norms.csv', delimiter=',')

In [28]:
def cleanjson(result):
    title = result[result.find("Title")+8:result.find("Year")-3]
    plot = result[result.find("Plot")+7:result.find("Language")-3]
    try:
        review_imdb = float(result[result.find(
        '"Internet Movie Database","Value":"')+35:result.find('Source":"Rotten Tomatoes"')-8])
    except:
        review_imdb = -1
    try:
        review_rotten = float(result[result.find(
        'Source":"Rotten Tomatoes","Value":')+35: result.find('},{"Source":"Metacritic"')-2])
    except:
        review_rotten = -1
    return [title, plot, review_imdb, review_rotten]

def response(result):
    text = result[result.find('"Response":' )+12:]
    return text.find('True') >-1

In [29]:
omdb_TOKEN = 'ce887dbd'
def find_movie(movie):
    title = ""
    plot = ""
    query = "http://www.omdbapi.com/?apikey=" + omdb_TOKEN + "&t=" + movie
    params = {"r": "json", "plot": "full"}
    result = requests.get(query, params)
    if response(result.text):
        json = cleanjson(result.text)
        plot = json[1]
        title = json[0]
        review_imdb = json[2]
        review_rotten = json[3]
    else:
        return "ERROR"
    return (title, plot, review_imdb, review_rotten)

In [30]:
def index_search(user_mov):
    movie = find_movie(user_mov)
    query = movie[1]
    #query = ''
    #if movie[0] in movie_titles:
    #    query = movies['Summary'][title_to_index[movie[0]]]
    #else:
    #    query = movie[1]
    scores = np.zeros(len(norms))
    docs = [i for i in range(len(norms))]
    q = query.lower()
    q_tokens = tokenizer(q)
    q_norm_sq = 0
    for t in set(q_tokens):
        if t in idf:
            q_norm_sq += (q_tokens.count(t)*idf[t])**2
            for (doc,cnt) in inv_idx[t]:
                scores[doc] += (q_tokens.count(t)*cnt*idf[t]**2)/norms[doc]
    q_norm = math.sqrt(q_norm_sq)
    new_scores = [score/q_norm for score in scores]
    result = sorted(tuple(zip(new_scores, docs)),reverse=True)
    return result[:10]

In [31]:
results = index_search('Memento')

In [32]:
results

[(0.16056939582947827, 35155),
 (0.15718873462135707, 13330),
 (0.15580892217745576, 37607),
 (0.1485379138540403, 2487),
 (0.14042652859711635, 34573),
 (0.13924014073325508, 17758),
 (0.13512155658997846, 15566),
 (0.1330696150046359, 3533),
 (0.13249725484059963, 41117),
 (0.12548895093423626, 2898)]

In [28]:
def get_10(movie,results):
    ten = []
    for sim, ind in results[:10]:
        if movies['Title'][ind] != movie:
            s = movies['Title'][ind]+' (score: '+str(sim)+')'
            ten.append(s)
    return ten

In [29]:
get_10('Memento',results)

['Frauds (score: 0.16056939582947824)',
 'Duel in the Jungle (score: 0.15718873462135707)',
 'The Fire Raisers (score: 0.15580892217745576)',
 'Cover Up (score: 0.1485379138540403)',
 "Killer's Carnival (score: 0.14042652859711635)",
 'Perdido por perdido (score: 0.13924014073325505)',
 'The Gentleman from Nowhere (score: 0.1351215565899785)',
 'Spring Fever (score: 0.1330696150046359)',
 'Adulterous Wife: Dizzy (score: 0.13249725484059963)',
 'Zombie Girl: The Movie (score: 0.12548895093423626)']

In [1]:
movies['Title'][35155]

NameError: name 'movies' is not defined

In [24]:
#id_to_index = {wikiid:i for i, wikiid in enumerate(movies['WikiID'])}
#title_to_id = {t:i for t, i in zip(movies['Title'], movies['WikiID'])}
#id_to_title = {v:k for k,v in title_to_id.items()}
#title_to_index = {t:i for i, t in enumerate(movies['Title'])}
#index_to_title = {v:k for k,v in title_to_index.items()}
#movie_titles = [t for t in movies['Title']]

In [31]:
doc_by_vocab = vectorizer.fit_transform([s for s in movies['Summary']]).toarray()

In [33]:
def get_summary(movie, data):
    wiki_id = title_to_id.get(movie)
    ind = id_to_index.get(wiki_id)
    summary = data['Summary'][ind]
    return summary

In [44]:
def get_sim(user_mov, other_mov):
    j = doc_by_vocab[title_to_index[other_mov]]
    result = find_movie(user_mov)
    if result[0] in movie_titles:
        i = doc_by_vocab[title_to_index[result[0]]]
    else:
        i = vectorizer.transform([result[1]]).toarray()[0]
    cosim = np.dot(i,j) / (np.linalg.norm(i)*np.linalg.norm(j))
    return cosim